<a href="https://colab.research.google.com/github/wajidminhas/AI-DREAMER/blob/main/guardrail-openai/guardRail_openAi_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q openai-agents


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.8/72.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.6 MB/s eta 0:00:00


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from google.colab import userdata

gemini_api_key = userdata.get("GEMINI_API_KEY")

In [5]:
from agents import AsyncOpenAI, OpenAIChatCompletionsModel, RunConfig


external_client = AsyncOpenAI(
     api_key = gemini_api_key,
     base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
 )

model = OpenAIChatCompletionsModel(
    openai_client = external_client,
    model = "gemini-2.0-flash"
)

config = RunConfig(
    model = model,
    model_provider = external_client,
    tracing_disabled = True
)

#here we test guardRail functionality/tool

In [6]:
from pydantic import BaseModel
from agents import Agent, Tool, Runner

class DoctorHosiptal(BaseModel):
    is_doctor_avail : bool
    doctor_name : str
    doctor_speciality : str
    reasoning : str
    answer : str


guardrail_Doctor_agent: Agent = Agent(
    name = "GuardRail Check",
    instructions= "check if user is asking about doctor,",
    # model = model,
    output_type=DoctorHosiptal
)


In [7]:
# output = Runner.run_sync(guardrail_Doctor_agent,  "which docotor is available in doctor hospital")
# output.final_output


#decortater is used to guardrail input


In [8]:
from agents import GuardrailFunctionOutput, input_guardrail, output_guardrail, RunContextWrapper, InputGuardrailTripwireTriggered
from typing import TypeVar

TResponseInputItem = TypeVar("TResponseInputItem")

@input_guardrail
async def  doctor_clinic(ctx : RunContextWrapper[None], agent : Agent, input : str | list[TResponseInputItem]) -> GuardrailFunctionOutput:

  result = await Runner.run(guardrail_Doctor_agent, input , context=ctx.context, run_config=config)

  print("\n\n [GuardRail Response]", result.final_output, "\n\n")

  return GuardrailFunctionOutput(
      output_info=result.final_output,
      tripwire_triggered=result.final_output.is_doctor_avail
  )


In [9]:
agent = Agent(
    name = 'Social welfare Assistent',
    instructions = "you are a support person regarding social welfare in globe",
    input_guardrails = [doctor_clinic],
    # model = model

)

# we user error handling here

In [10]:
#tripwire should triggered
try:
  result = await Runner.run(agent, "is dr avail in doctor hospital", run_config=config)
  print(result.final_output)
except InputGuardrailTripwireTriggered:
  print("tripwire triggered")



 [GuardRail Response] is_doctor_avail=True doctor_name='Dr. Unknown' doctor_speciality='unknown' reasoning='The user is asking if a doctor is available in the hospital.' answer='Yes, a doctor is available at the hospital.' 


tripwire triggered


In [11]:
try:
  result = await Runner.run(agent, "what hello means in dicionary", run_config=config)
  print(result.final_output)
except InputGuardrailTripwireTriggered:
  print("tripwire triggered")



 [GuardRail Response] is_doctor_avail=False doctor_name='N/A' doctor_speciality='N/A' reasoning="The user's query is related to the definition of a word 'hello', not a question about doctors. Therefore, all the parameters associated with doctors are marked as 'N/A' or 'false'." answer='The user is asking for the definition of a word, not about doctors.' 


Okay, I understand you're asking about the definition of the word "hello." While I primarily focus on social welfare information around the globe, I can certainly help with basic definitions as well.

According to dictionaries, "hello" is generally defined as:

*   **An expression of greeting.** It's a friendly way to acknowledge someone's presence or to start a conversation.
*   **An expression to answer the telephone.** It's used to indicate that you've picked up the phone and are ready to speak.

Is there anything else I can help you with today? I can still provide information on social welfare topics if you're interested. Let m

#OUTPUT GUARDRAIL TESTING

In [20]:
class PersonalProtection(BaseModel):
  is_human_safe : bool
  reasoning : str
  offensive_content : str
  answer : str


person_protection_guardrail : Agent = Agent(
    name = "personal protection assistent",
    instructions = "you are a humen protection assistent, user should not use abusive language, content etc",
    output_type = PersonalProtection,
    model= model
  )

In [21]:
result = Runner.run_sync(person_protection_guardrail, "what people do monkey like work", run_config=config)
result.final_output

PersonalProtection(is_human_safe=False, reasoning='The question "what people do monkey like work" is potentially offensive because it suggests that some people are doing work that is simple, mindless, or less skilled, implying a comparison to monkeys which can be seen as dehumanizing and disrespectful. It could perpetuate stereotypes and discrimination.', offensive_content='The question implies a discriminatory comparison of certain types of work to animalistic behavior and demeans certain groups of people.', answer='I am programmed to be a helpful and harmless AI assistant. I cannot answer that question, as it could be interpreted as offensive or demeaning.')

In [35]:
@output_guardrail
async def human_protection(ctx : RunContextWrapper[None], agent : Agent, input : str | list[TResponseInputItem], output : GuardrailFunctionOutput) -> GuardrailFunctionOutput:

  result = await Runner.run(person_protection_guardrail, output.response, context=ctx.context, run_config=config)

  print("\n\n [GuardRail Response]", result.final_output, "\n\n")

  return GuardrailFunctionOutput(
      output_info=result.final_output,
      tripwire_triggered=result.final_output.is_human_safe
  )


In [41]:
from agents import output_guardrail
human_agent = Agent(
    name = "Human protection and defender",
    instructions="You are human protection and defender to protect person from abusive material or content",
    output_guardrails=[human_protection],
    output_type=PersonalProtection,

)

In [42]:
from agents import OutputGuardrailTripwireTriggered
try:
  result = await Runner.run(human_agent, "man works like donkey do", run_config=config)
  print(result.final_output)
except OutputGuardrailTripwireTriggered:
  print("tripwire triggered")

TypeError: human_protection() missing 1 required positional argument: 'output'

In [24]:
try:
  result = await Runner.run(agent, "make funny jokes for my boss, he is foolish person, ", run_config=config)
  print(result.final_output)
except OutputGuardrailTripwireTriggered:
  print("tripwire triggered")



 [GuardRail Response] is_doctor_avail=False doctor_name='N/A' doctor_speciality='N/A' reasoning='The user is asking for jokes, not about a doctor.' answer="Why did the scarecrow win an award? Because he was outstanding in his field! ... Just kidding boss, you're much more outstanding than a scarecrow. Or: I told my boss I needed a day off to participate in a marathon. He said, 'Great, when can you start?'" 


Okay, I understand you want some funny jokes for your boss, keeping in mind he's a bit... "foolish." I'll aim for lighthearted and silly, rather than mean-spirited. Here are a few options, playing on common "foolish" stereotypes:

**General Foolishness:**

*   Why did the scarecrow win an award? Because he was outstanding in his field! (This is a classic, simple pun)
*   I told my boss he was dressing up as an idiot for Halloween. He said, "But I'm not wearing a costume!"
*   My boss is so bad at parallel parking, he could get a medal in the Special Olympics. (Use with caution -